#AirBnB Project

##Loading the libraries

In [ ]:
library(dplyr)
library(tidyr)
library(ggplot2)
library(data.table)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




##Loading the data and looking at the first few rows


In [ ]:
df = read.csv('Airbnb_Open_Data.csv')
head(df)

,id,NAME,host.id,host_identity_verified,host.name,neighbourhood.group,neighbourhood,lat,long,country,⋯,service.fee,minimum.nights,number.of.reviews,last.review,reviews.per.month,review.rate.number,calculated.host.listings.count,availability.365,house_rules,license
,<int>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,⋯,<chr>,<int>,<int>,<chr>,<dbl>,<int>,<int>,<int>,<chr>,<chr>
1,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,⋯,$193,10,9,10/19/2021,0.21,4,6,286,Clean up and treat the home the way you'd like your home to be treated. No smoking.,
2,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,⋯,$28,30,45,5/21/2022,0.38,4,2,228,Pet friendly but please confirm with me if the pet you are planning on bringing with you is OK. I have a cute and quiet mixed chihuahua. I could accept more guests (for an extra fee) but this also needs to be confirmed beforehand. Also friends traveling together could sleep in separate beds for an extra fee (the second bed is either a sofa bed or inflatable bed). Smoking is only allowed on the porch.,
3,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,,Elise,Manhattan,Harlem,40.80902,-73.94190,United States,⋯,$124,3,0,,NA,5,1,352,"I encourage you to use my kitchen, cooking and laundry facilities. There is no additional charge to use the washer/dryer in the basement. No smoking, inside or outside. Come home as late as you want. If you come home stumbling drunk, it's OK the first time. If you do it again, and you wake up me or the neighbors downstairs, we will be annoyed. (Just so you know . . . )",
4,1002755,,85098326012,unconfirmed,Garry,Brooklyn,Clinton Hill,40.68514,-73.95976,United States,⋯,$74,30,270,7/5/2019,4.64,4,1,322,,
5,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,⋯,$41,10,9,11/19/2018,0.10,3,1,289,"Please no smoking in the house, porch or on the property (you can go to the nearby corner). Reasonable quiet after 10:30 pm. Please remove shoes in the house.",
6,1004098,Large Cozy 1 BR Apartment In Midtown East,45498551794,verified,Michelle,Manhattan,Murray Hill,40.74767,-73.97500,United States,⋯,$115,3,74,6/22/2019,0.59,3,1,374,"No smoking, please, and no drugs.",


##Data Cleaning
###Firstly, we looked at how many NA values each column had

In [ ]:
df %>% is.na %>% colSums %>% as.data.frame

,.
,<dbl>
id,0
NAME,0
host.id,0
host_identity_verified,0
host.name,0
neighbourhood.group,0
neighbourhood,0
lat,8
long,8


##Stripped the dollar sign off the price columns and converted it into numeric value



In [ ]:
df$price_cleaned = gsub("\\$", "", df$price)
df$price_cleaned = gsub(",", "", df$price_cleaned)
df$price_numeric = as.numeric(df$price_cleaned)

###247 values remained unconverted so we dropped those


In [ ]:
df$price_numeric %>% is.na %>% sum

[1] 247

In [ ]:
df[is.na(df$price_numeric), 'price'] %>% table

.
    
247 

In [ ]:
df = df[!is.na(df$price_numeric), ]

In [ ]:
df$price = df$price_numeric
df$price_numeric = NULL

In [ ]:
names(df)

[1] "id"                             "NAME"                          
 [3] "host.id"                        "host_identity_verified"        
 [5] "host.name"                      "neighbourhood.group"           
 [7] "neighbourhood"                  "lat"                           
 [9] "long"                           "country"                       
[11] "country.code"                   "instant_bookable"              
[13] "cancellation_policy"            "room.type"                     
[15] "Construction.year"              "price"                         
[17] "service.fee"                    "minimum.nights"                
[19] "number.of.reviews"              "last.review"                   
[21] "reviews.per.month"              "review.rate.number"            
[23] "calculated.host.listings.count" "availability.365"              
[25] "house_rules"                    "license"

##Count of Neighborhood Group
### First we had to correct the mistyped values of "manhatan" and "brookln" to match the correct spelling. We also made the blank values have a "-" name. Then we could show the amount of AirBnBs in each neighborhood group



In [ ]:
table(df$neighbourhood.group)


                      Bronx       brookln      Brooklyn      manhatan 
           28          2705             1         41749             1 
    Manhattan        Queens Staten Island 
        43682         13234           952 

In [ ]:
df[df$neighbourhood.group == "manhatan", 'neighbourhood.group'] = "Manhattan"
df[df$neighbourhood.group == "brookln", 'neighbourhood.group'] = "Brooklyn"
df[df$neighbourhood.group == "", 'neighbourhood.group'] = "-"


In [ ]:
table(df$neighbourhood.group)


            -         Bronx      Brooklyn     Manhattan        Queens 
           28          2705         41750         43683         13234 
Staten Island 
          952 

##Averages
### Here the room types and the locations of the AirBnBs are grouped and their averages are calculated. After cleaning up the price column by stripping away the dollar and comma, as well as geetting rid of null values, we can compute this average.



In [ ]:
df %>% 
  group_by(neighbourhood.group, room.type) %>% 
  summarise(mean(price))

`summarise()` has grouped output by 'neighbourhood.group'. You can override
using the `.groups` argument.


neighbourhood.group,room.type,mean(price)
<chr>,<chr>,<dbl>
-,Entire home/apt,692.8000
-,Private room,639.2222
Bronx,Entire home/apt,620.2326
Bronx,Private room,634.7106
Bronx,Shared room,600.2564
Brooklyn,Entire home/apt,626.8964
Brooklyn,Hotel room,736.1250
Brooklyn,Private room,625.8685
Brooklyn,Shared room,634.3171


In [ ]:
df %>% group_by(review.rate.number) %>% summarise(mean(price))

review.rate.number,mean(price)
<int>,<dbl>
1,519.6876
2,525.2879
3,523.5136
4,527.0568
5,525.2491
NA,499.5240


In [ ]:
mean(df$price)


[1] 524.7024

##House rules
### We added 2 columns to the data frame that will tell us if the smoking and pets are allowed in the airbnb. We will use grep function to select all the rows (airbnb listings) in the data frame that contain "no smoking" and "no pets" in their house rules. Then we set those rows as a 1 to represent that in those airbnbs, pets and smoking are not allowed. In case they are allowed, 0 was assigned.



In [ ]:
df$no_smoking=NA
df$no_pets=NA
df[grep("no smoking",df$house_rules),"no_smoking"]=1
df[grep("no pets",df$house_rules),"no_pets"] =1
df<-df %>% 
  mutate_at(c('no_smoking','no_pets'), ~replace_na(.,0))

write.csv(df,"C:/Users/msoric/Downloads/Airbnb_clean_data.csv")


The updated data frame was exported and used in Tableu for further analysis. 
